In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
work_path = '/home/mmde-lab/s3/59e875d818c7b/'
order_path = 'kddi_r_pay_order.tsv.gz'
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')

In [3]:
df_order.ins_tm.min(), df_order.ins_tm.max()

('2016-10-01 00:00:01', '2017-09-30 23:59:58')

In [4]:
df_order['ins_tm'] = pd.to_datetime(df_order['ins_tm'],format='%Y-%m-%d %H:%M:%S')

In [5]:
df_order['date'] = df_order.ins_tm.dt.date

In [6]:
train_order = df_order.loc[df_order.date <= datetime.date(2017,9,14)]

In [7]:
valid_order = df_order.loc[df_order.date >= datetime.date(2017,9,15)]
valid_order = valid_order.loc[df_order.date <= datetime.date(2017,9,23)]

In [8]:
test_order = df_order.loc[df_order.date >= datetime.date(2017,9,24)]

In [9]:
# Get corpus from trainingset 
train_order.deal_id = train_order.deal_id.map(lambda x: str(x))
grouped_order = train_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = True)
Input_Data = Input_Data.iloc[:,0].values
corpus = []
for line in Input_Data:
    words = [x for x in line.split(' ')]
    corpus.append(words)
np.save('session_based_corpus_trainset.npy', corpus)

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim import corpora
model = Word2Vec.load('/tmp/session_based_trainset_word2vec_model')
corpus = np.load('session_based_corpus_trainset.npy')
dictionary = corpora.Dictionary(corpus)
dictionary.filter_extremes(no_below=1, no_above=1, keep_n=300000, keep_tokens=None)
corpus_dict = [dictionary.doc2bow(text) for text in corpus]
tfidf = TfidfModel(corpus_dict)

In [11]:
sell_count = train_order.deal_id.value_counts()
sell = pd.DataFrame(sell_count)
sell = sell.rename(columns= {'deal_id':'sell_count'})
sell = sell.reset_index()
sell.columns = ['deal_id','sell_count']
def calcu_item2vec(raw_des):
    itemvec = np.zeros([1,200], dtype = float)
    itemvec = model[str(raw_des)]
    return itemvec
sell['dealvec'] = sell.deal_id.map(lambda x: calcu_item2vec(x))

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [12]:
sell.to_csv('df_item_sbwin5_trainset.csv.gz', compression = 'gzip')

In [13]:
train_order.deal_id = train_order.deal_id.map(lambda x: str(x))
grouped_order = train_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = False)
Input_Data.columns = ['account_id', 'deal_id']
df_user_sbwin5 = Input_Data
df_user_sbwin5.deal_id = df_user_sbwin5.deal_id.map(lambda x: x.split(' '))
def calcu_user2vec(raw_des):
    user_corpus = dictionary.doc2bow(raw_des)
    uservec = np.zeros([1,200], dtype = float)
    for j in range(len(user_corpus)):
        uservec += tfidf[user_corpus][j][1]*model[dictionary.get(user_corpus[j][0])]
    return uservec
df_user_sbwin5['uservec'] = df_user_sbwin5.deal_id.map(lambda x: calcu_user2vec(x))
df_user_sbwin5.to_csv('df_user_sbwin5_trainset.csv.gz', compression = 'gzip')

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
